In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import requests
import json

random_state = 42

In [2]:
raw_data = pd.merge(pd.read_csv('../../anomaly/data/train_transaction.csv'), pd.read_csv('../../anomaly/data/train_identity.csv'),
                    left_on='TransactionID', right_on='TransactionID', how='left')

X_train, X_test = train_test_split(raw_data, test_size=0.2, random_state=random_state)
X_train.reset_index(drop=True, inplace=True)
y_train = X_train['isFraud']
X_train.drop(columns=['isFraud', 'TransactionID'], inplace=True)

X_test.reset_index(drop=True, inplace=True)
y_test = X_test['isFraud']
X_test.drop(columns=['isFraud', 'TransactionID'], inplace=True)

del raw_data

### Запрос для одного объекта

Возьмём 50 мошеннических транзакций для тестирования сервиса

In [3]:
df_req = X_test[y_test==1].sample(50, random_state=random_state)
df_req.reset_index(drop=True, inplace=True)

Для каждой транзакции выполним отдельный запрос

In [4]:
%%time
url = 'http://127.0.0.1:8000/predict_one_transaction'
api_single_res = []
for i in range(df_req.shape[0]):
    response = requests.post(url, data=json.dumps(df_req.loc[i].to_dict()), headers={"Content-Type": "application/json"})
    api_single_res.append(response.json())

CPU times: total: 250 ms
Wall time: 4.59 s


Результаты запросов: 0 - транзакция не мошенническая, 1 - транзакция мошенническая

In [5]:
sum(api_single_res)

34.0

In [6]:
print(api_single_res)

[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]


### Запрос для списка транзакций

In [7]:
url_items = 'http://127.0.0.1:8000/predict_transactions'
lst = []
for i in range(df_req.shape[0]):
    lst.append(df_req.loc[i].to_dict())
response_items = requests.post(url_items, data=json.dumps(lst), headers={"Content-Type": "application/json"})
print(response_items.status_code)

200


In [8]:
sum(response_items.json())

34.0

In [9]:
print(response_items.json())

[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]


### Запрос для тестирования панели 'Total Exception' дашборда 'FastAPI Service' в Grafana.
По этому запросу сервис ничего не делает. Только бросает исключение

In [10]:
url_raise_exception = 'http://127.0.0.1:8000/raise_exception'
for i in range(50):
    resp_raise_exception = requests.get(url_raise_exception)
resp_raise_exception.status_code

500

### Grafana

Посмотрим на дашборд 'FastAPI Service'

Grafana http://localhost:3000/

<center>
<img src="grafana_dashboard.JPG"> 
</center>